In [272]:
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import numpy as np
from datetime import date

pd.set_option('display.max_rows', 100) 
pd.set_option('display.max_columns', 20) 

In [273]:
#Create empty lists for our contract extension data
extension_player_names = []
extension_details =  []
extension_dates = []
extension_types = []
extension_amounts = []
extension_lengths = []

#Create empty lists for our free agent data
fa_player_names = []
fa_details =  []
fa_dates = []
fa_types = []
fa_amounts = []
fa_length = []

In [274]:
#Create a function to pull all of our player names, transaction dates, and contract info from the HTML page
def data_parse(file_type, filepath, dates_list, types_list, amounts_list, length_list, names_list):
    with open(filepath) as htmlfile:
        data = htmlfile.read()
        soup = BeautifulSoup(data, 'html')
        content = soup.find_all("small", class_ = "d-block")

        for detail in content:
            strong = detail.find("strong")
            if strong:
                date = strong.text.strip()
                full_text = detail.get_text(strip=True)
                full_detail = full_text.replace(strong.text, "").strip("-")
                type_range_start = full_detail.find("million")
                type_range_end = full_detail.find("contract")
                amount_range_start = full_detail.find("$")+1
                amount_range_end = type_range_start-1
                length = full_detail.find("year")-2

                dates_list.append(date)
                if file_type == 'extension':
                    types_list.append(full_detail[type_range_start+8:type_range_end])
                else:
                    types_list.append('Free Agent')
                amounts_list.append(full_detail[amount_range_start:amount_range_end])
                length_list.append(full_detail[length])

        all_extension_names = soup.find_all(class_ = "text-danger h4")
        for name in all_extension_names:
            names_list.append(name.string)
        
    extension_data = pd.DataFrame({'Player Name':pd.Series(names_list),
                                'Date':pd.Series(dates_list),
                                'Contract Type':pd.Series(types_list),
                                'Length (years)':pd.Series(length_list),
                                'Total Amount ($ million)':pd.Series(amounts_list)})
    
    return(extension_data)

#Define filepaths
extension_filepath = "signed-extended.html"
fa_filepath = "signed-fa.html"

#Apply function to extension and free agent signee data
extensions = data_parse('extension', extension_filepath, extension_dates, extension_types, extension_amounts,extension_lengths, extension_player_names)
free_agents = data_parse('free_agents', fa_filepath, fa_dates, fa_types, fa_amounts, fa_length, fa_player_names)

In [275]:
#Combine free agents & extensions data
all_data = pd.concat([free_agents, extensions])

#Remove coaches from the dataset
all_data = all_data[~all_data['Player Name'].str.contains("(COA)")]

#Remove position labels
all_data['Player Name'] = all_data['Player Name'].str.replace(r'\s\([A-Z]+\)$', '', regex=True)

#Set player names as index to make data updates easier
all_data.set_index('Player Name', inplace = True)

#Data Cleanup where there are typos or missing information in Spotrac's contract details
all_data.loc['Luka Garza','Total Amount ($ million)'] = '5.26'
all_data.loc['Shai Gilgeous-Alexander','Contract Type'] = 'Supermax Extension'
all_data.loc['Devin Booker','Contract Type'] = 'Supermax Extension'
all_data.loc['Devin Booker','Total Amount ($ million)'] = '133.25'

#Remove players without contract amount details
all_data.drop(['Jock Landale'], axis = 0, inplace=True)

#Convert contract amounts and lengths to floats
all_data['Total Amount ($ million)'] = all_data['Total Amount ($ million)'].astype(float)
all_data['Length (years)'] = all_data['Length (years)'].astype(float)

#Create a copy of the data to be used for a separate analysis (total contract values negotiated by agency)
all_free_agency_data = all_data.copy()

/var/folders/dx/8d37nn691w30px6b593c5y040000gn/T/ipykernel_49232/2101599881.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  all_data = all_data[~all_data['Player Name'].str.contains("(COA)")]


In [276]:
#Create a list of salary years
seasons = ['salary_2026', 'salary_2027', 'salary_2028', 'salary_2029', 'salary_2030', 'salary_2031']

#Calculate average salary (without rounding) for all_data
all_data['Average Salary'] = all_data['Total Amount ($ million)'] / all_data['Length (years)']

#Calculate average salary (with rounding) for all_free_agency data
all_free_agency_data['Average Salary'] = round(all_free_agency_data['Total Amount ($ million)'] / all_free_agency_data['Length (years)'],1)

#Create empty data columns for future annual salary as a peercentage of the cap
all_data['salary_2025-26'], all_data['salary_2026-27'], all_data['salary_2027-28'], all_data['salary_2028-29'], all_data['salary_2029-30'], all_data['salary_2030-31'] = np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN

#Calculate salary as a percentage of the future cap for all free agents, with growing denominators to adjust for cap projections (which all will begin in the 2025-26 season)
all_data['salary_2025-26'] = np.where((all_data['Contract Type']=='Free Agent') & (all_data['Length (years)'] >= 1), round(all_data['Average Salary']/154.647,4), np.NaN)
all_data['salary_2026-27'] = np.where((all_data['Contract Type']=='Free Agent') & (all_data['Length (years)'] >= 2), round(all_data['Average Salary']/165.465,4), np.NaN)
all_data['salary_2027-28'] = np.where((all_data['Contract Type']=='Free Agent') & (all_data['Length (years)'] >= 3), round(all_data['Average Salary']/182.019,4), np.NaN)
all_data['salary_2028-29'] = np.where((all_data['Contract Type']=='Free Agent') & (all_data['Length (years)'] >= 4), round(all_data['Average Salary']/191.120,4), np.NaN)
all_data['salary_2029-30'] = np.where((all_data['Contract Type']=='Free Agent') & (all_data['Length (years)'] >= 5), round(all_data['Average Salary']/200.651,4), np.NaN)

#Round average salary for all_data to 2 decimals (now that computation is finished)
all_data['Average Salary'] = round(all_data['Total Amount ($ million)'] / all_data['Length (years)'],2)

all_data[all_data['Contract Type']== 'Free Agent'].head()

,Date,Contract Type,Length (years),Total Amount ($ million),Average Salary,salary_2025-26,salary_2026-27,salary_2027-28,salary_2028-29,salary_2029-30,salary_2030-31
Player Name,,,,,,,,,,,
Jordan McLaughlin,"Jul 10, 2025",Free Agent,1.0,2.87,2.87,0.0186,NaN,NaN,NaN,NaN,NaN
Ryan Rollins,"Jul 09, 2025",Free Agent,3.0,12.00,4.00,0.0259,0.0242,0.022,NaN,NaN,NaN
Jericho Sims,"Jul 09, 2025",Free Agent,2.0,5.26,2.63,0.0170,0.0159,NaN,NaN,NaN,NaN
Nigel Hayes-Davis,"Jul 09, 2025",Free Agent,1.0,2.05,2.05,0.0133,NaN,NaN,NaN,NaN,NaN
Bruce Brown Jr.,"Jul 09, 2025",Free Agent,1.0,3.08,3.08,0.0199,NaN,NaN,NaN,NaN,NaN


In [277]:
#Manually calculate salary as a percentage of the future cap for all players that signed extensions (which start at different time windows depending on the contract situation)
all_data.loc['Jakob Poeltl', 'salary_2027-28':'salary_2029-30'] = [round(26.080247/165.465,4), round(28.166667/182.019,4), round(30.253086/191.120,4)]
all_data.loc['Jaren Jackson Jr.', 'salary_2025-26':'salary_2029-30'] = [round(48.00/154.647,4), round(48.00/165.465,4), round(48.00/182.019,4), round(48.00/191.120,4), round(48.00/200.651,4)]
all_data.loc['Shai Gilgeous-Alexander', 'salary_2027-28':'salary_2030-31'] = [round(63.706650/182.019,4), round(68.803182/191.120,4), round(73.899714/200.651,4), round(78.996246/210.713,4)]
all_data.loc['Jaylin Williams', 'salary_2025-26':'salary_2027-28'] = [round(8.45/154.647,4), round(7.74/165.465,4), round(7.74/182.019,4)]
all_data.loc['Jabari Smith Jr.', 'salary_2026-27':'salary_2030-31'] = [round(24.40/165.465,4), round(24.20/182.019,4), round(24.40/191.120,4), round(24.40/200.651,4), round(24.40/210.713,4)]
all_data.loc['Paolo Banchero', 'salary_2026-27':'salary_2030-31'] = [round(41.368/165.465,4), round(44.67744/182.019,4), round(47.98688/191.120,4), round(51.296320/200.651,4), round(54.605760/210.713,4)]
all_data.loc['Chet Holmgren', 'salary_2026-27':'salary_2030-31'] = [round(41.368/165.465,4), round(44.67744/182.019,4), round(47.98688/191.120,4), round(51.296320/200.651,4), round(54.605760/210.713,4)]
all_data.loc['Daniel Gafford', 'salary_2026-27':'salary_2028-29'] = [round(17.263584/165.465,4), round(18.126763/182.019,4), round(18.989942/191.120,4)]
all_data.loc['Steven Adams', 'salary_2025-26':'salary_2027-28'] = [round(17.263584/154.647,4), round(18.126763/165.465,4), round(18.989942/182.019,4)]
all_data.loc['Devin Booker', 'salary_2028-29':'salary_2029-30'] = [round(64.065951/182.019,4), round(69.191227/191.120,4)]

all_data[all_data['Contract Type']!= 'Free Agent']

,Date,Contract Type,Length (years),Total Amount ($ million),Average Salary,salary_2025-26,salary_2026-27,salary_2027-28,salary_2028-29,salary_2029-30,salary_2030-31
Player Name,,,,,,,,,,,
Devin Booker,"Jul 09, 2025",Supermax Extension,2.0,133.25,66.62,NaN,NaN,NaN,0.3520,0.3620,NaN
Chet Holmgren,"Jul 09, 2025",rookie scale maximum,5.0,239.93,47.99,NaN,0.2500,0.2455,0.2511,0.2556,0.2591
Paolo Banchero,"Jul 07, 2025",Rookie Scale,5.0,239.93,47.99,NaN,0.2500,0.2455,0.2511,0.2556,0.2591
Jakob Poeltl,"Jul 07, 2025",veteran,3.0,84.50,28.17,NaN,NaN,0.1576,0.1547,0.1583,NaN
Shai Gilgeous-Alexander,"Jul 07, 2025",Supermax Extension,4.0,272.43,68.11,NaN,NaN,0.3500,0.3600,0.3683,0.3749
Jabari Smith Jr.,"Jul 06, 2025",Rookie Scale,5.0,122.00,24.40,NaN,0.1475,0.1330,0.1277,0.1216,0.1158
Jaren Jackson Jr.,"Jun 30, 2025",Renegotiation-and-Extend,5.0,240.00,48.00,0.3104,0.2901,0.2637,0.2512,0.2392,NaN
Jaylin Williams,"Jun 29, 2025",veteran,3.0,23.93,7.98,0.0546,0.0468,0.0425,NaN,NaN,NaN
Daniel Gafford,"Jun 23, 2025",veteran,3.0,54.38,18.13,NaN,0.1043,0.0996,0.0994,NaN,NaN


In [278]:
#Load ESPN tabular 2024-25 salary data
salary_data = pd.read_csv('player-salaries.csv')
salary_data.head()

,RK,NAME,TEAM,SALARY
0,1,Stephen Curry,Golden State Warriors,55761216
1,2,Joel Embiid,Philadelphia 76ers,51415938
2,3,Nikola Jokic,Denver Nuggets,51415938
3,4,Kevin Durant,Phoenix Suns,51179021
4,5,Bradley Beal,Phoenix Suns,50203930


In [279]:
#Drop unnecessary columns
salary_data = salary_data.drop(['RK','TEAM'], axis=1)

#Rename 'SALARY' column
salary_data.rename(columns={'SALARY':'current_salary ($ million)'}, inplace=True)

#Calculate current salary as a percentage of the 2024-25 salary cap
salary_data['current_cap_pct'] = round(salary_data['current_salary ($ million)']/140588000,4)

#Round salary dollar value to 2 decial places
salary_data['current_salary ($ million)'] = round(salary_data['current_salary ($ million)']/1000000,2)

#Clean up player names and use NAME as the df index
salary_data['NAME'] = np.where(salary_data['NAME']=='Bruce Brown', "Bruce Brown Jr.", salary_data['NAME'])
salary_data['NAME'] = np.where(salary_data['NAME']=='Dennis Schroder', "Dennis Schröder", salary_data['NAME'])
salary_data['NAME'] = np.where(salary_data['NAME']=="Jae'Sean Tate", "Jae’Sean Tate", salary_data['NAME'])
salary_data.set_index('NAME', inplace=True)

#Join the free agent signee salary data to the list of 2024-25 players & salaries
all_salary_data = pd.merge(all_data, salary_data, how='left', left_index = True, right_index = True)

#Set the salary value players who did not count against the 2024-25 salary cap as zero
all_salary_data.loc['Cam Spencer', ['current_salary ($ million)', 'current_cap_pct']] = [0.00, 0.0000]
all_salary_data.loc['Nigel Hayes-Davis', ['current_salary ($ million)', 'current_cap_pct']] = [0.00, 0.0000]
all_salary_data.loc['Collin Gillespie', ['current_salary ($ million)', 'current_cap_pct']] = [0.00, 0.0000]

all_salary_data.head()

,Date,Contract Type,Length (years),Total Amount ($ million),Average Salary,salary_2025-26,salary_2026-27,salary_2027-28,salary_2028-29,salary_2029-30,salary_2030-31,current_salary ($ million),current_cap_pct
Player Name,,,,,,,,,,,,,
Jordan McLaughlin,"Jul 10, 2025",Free Agent,1.0,2.87,2.87,0.0186,NaN,NaN,NaN,NaN,NaN,2.09,0.0148
Ryan Rollins,"Jul 09, 2025",Free Agent,3.0,12.00,4.00,0.0259,0.0242,0.022,NaN,NaN,NaN,0.49,0.0035
Jericho Sims,"Jul 09, 2025",Free Agent,2.0,5.26,2.63,0.0170,0.0159,NaN,NaN,NaN,NaN,2.09,0.0149
Nigel Hayes-Davis,"Jul 09, 2025",Free Agent,1.0,2.05,2.05,0.0133,NaN,NaN,NaN,NaN,NaN,0.00,0.0000
Bruce Brown Jr.,"Jul 09, 2025",Free Agent,1.0,3.08,3.08,0.0199,NaN,NaN,NaN,NaN,NaN,23.00,0.1636


In [280]:
#Calculate the average salary as a percentage of the salary cap for free agent signees across the life of their new contracts
all_salary_data['new_avg_cap_pct'] = round(all_salary_data[['salary_2025-26','salary_2026-27','salary_2027-28','salary_2028-29','salary_2029-30','salary_2030-31']].mean(axis=1),4)

#Calculate the difference between their 2024-25 salary as a percentage of the salary cap and their new salary as a percentage of the salary cap
all_salary_data['difference'] = all_salary_data['new_avg_cap_pct'] - all_salary_data['current_cap_pct']

#Sort the data by largest difference and drop future annual salary columns
all_salary_data.sort_values(by='difference', ascending=False, inplace=True)
all_salary_data = all_salary_data.drop(['salary_2025-26','salary_2026-27','salary_2027-28','salary_2028-29','salary_2029-30','salary_2030-31'], axis=1)

#Create a copy of the dataframe for to be used for a separate analysis (salaries vs. advanced metrics)
free_agent_salaries = all_salary_data.copy()

#Drop Rookie Scale Extensions and Buyouts from all_salary_data
all_salary_data.drop(['Deandre Ayton','Jabari Smith Jr.', 'Paolo Banchero', 'Chet Holmgren'], axis=0, inplace=True)

#Limit analaysis to the top 10 and bottom 10 in the 'difference' column
all_salary_data = pd.concat([all_salary_data.head(10), all_salary_data.tail(10)],axis=0)

print('all_salary_data', all_salary_data.head(5))

all_salary_data                                   Date              Contract Type  \
Player Name                                                         
Shai Gilgeous-Alexander   Jul 07, 2025         Supermax Extension   
Jaren Jackson Jr.         Jun 30, 2025  Renegotiation-and-Extend    
Santi Aldama              Jun 30, 2025                 Free Agent   
Nickeil Alexander-Walker  Jul 06, 2025                 Free Agent   
Luke Kornet               Jul 07, 2025                 Free Agent   

                          Length (years)  Total Amount ($ million)  \
Player Name                                                          
Shai Gilgeous-Alexander              4.0                    272.43   
Jaren Jackson Jr.                    5.0                    240.00   
Santi Aldama                         3.0                     52.50   
Nickeil Alexander-Walker             4.0                     62.00   
Luke Kornet                          4.0                     40.70   

         

In [281]:
#Load player agency representation data
agent_data = pd.read_csv('players-and-agents (agg).csv')
agent_data.head()

,Player,Team,Pos,Height,Weight,Birth Date,Age,Agency,Agent,Contractual Status,YOS
0,Max Abmas,Salt Lake City Stars,PG,5-11,162.0,"Apr 2, 2001",24.0,Wasserman,Josh Beauregard-Bell,Unrestricted Free Agent,0.0
1,Precious Achiuwa,New York Knicks,PF,6-8,243.0,"Sep 19, 1999",25.0,Octagon,Christopher Gaston\nAlex Saratsis,"New York Knicks, Jul 30, 2024",4.0
2,Steven Adams,Houston Rockets,C,6-11,265.0,"Jul 20, 1993",31.0,Wasserman,Darren Matsubara,"Houston Rockets, Jun 14, 2025",11.0
3,Bam Adebayo,Miami Heat,C-F,6-9,255.0,"Jul 18, 1997",27.0,Octagon,Alex Saratsis,"Miami Heat, Jul 6, 2024",7.0
4,Ochai Agbaji,Toronto Raptors,SG,6-5,215.0,"Apr 20, 2000",25.0,Octagon,Alex Saratsis,"Toronto Raptors, Jul 2, 2022",2.0


In [282]:
#Remove duplicate 'Jaylin Williams' from the dataset
agent_data = agent_data[agent_data['Team'] != 'Winnipeg Sea Bears']

#Keep essential columns only
agent_data = agent_data[['Player', 'Agency']]

#Join 'all_free_agency_data' with our player agency representation data on Player Name (and make player name the df index)
all_agency_data = pd.merge(all_free_agency_data, agent_data, how='left', left_index=True, right_on='Player')
all_agency_data.set_index('Player', inplace = True)

#Group the data by agency, and calculate the number of clients per agency to sign a contract and sum of contract dollars
aggregate = all_agency_data.groupby(by='Agency').agg({'Length (years)':['count','sum'],'Total Amount ($ million)':'sum'}).reset_index()

#Remove MultiIndex from columns
aggregate.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in aggregate.columns]

#Rename columns for readability - rename Agency to player as an alias to have rows in the player-level dataset represent the total contracts negotiated by agency
aggregate.rename(columns={'Agency_':'Player', 'Length (years)_count':'Contract Count', 'Length (years)_sum':'Length (years)' ,'Total Amount ($ million)_sum':'Total Amount ($ million)'}, inplace=True)
aggregate

,Player,Contract Count,Length (years),Total Amount ($ million)
0,AMR Agency,2,4.0,24.00
1,CAA Sports,13,32.0,755.74
2,Comsport,1,2.0,11.48
3,Equity Basketball,2,5.0,93.50
4,Excel Sports Management,5,14.0,267.67
5,Gersh Sports,1,4.0,40.70
6,Glushon Sports Management,6,9.0,65.60
7,Hazan Sports Management,1,3.0,24.00
8,Klutch Sports Group,6,13.0,129.79
9,LIFT Sports Management,5,15.0,394.98


In [283]:

#Prep data for UNION with player-level data by adding missing columns
aggregate['Agency'] = aggregate['Player']
aggregate['Contract Type'] = 'Agency Total'
aggregate['Average Salary'] = round(aggregate['Total Amount ($ million)'] / aggregate['Length (years)'], 1)
aggregate['Date'] = np.NaN
aggregate['Player'] = np.where(aggregate['Player'] == 'Shai Gilgeous-Alexander', 'Shai Gilgeous-Alexander MGMT', aggregate['Player'])
aggregate.set_index('Player', inplace=True)
new_order = ['Date', 'Contract Type', 'Length (years)', 'Total Amount ($ million)', 'Average Salary', 'Agency']
aggregate = aggregate[new_order]

#Union player agency / contract value data with agency totals and reset the index
new_agency_data = pd.concat([all_agency_data, aggregate])
new_agency_data.reset_index(inplace=True)
pd.set_option('display.max_rows', 10)
new_agency_data

,Player,Date,Contract Type,Length (years),Total Amount ($ million),Average Salary,Agency
0,Jordan McLaughlin,"Jul 10, 2025",Free Agent,1.0,2.87,2.9,Wasserman
1,Ryan Rollins,"Jul 09, 2025",Free Agent,3.0,12.00,4.0,Equity Basketball
2,Jericho Sims,"Jul 09, 2025",Free Agent,2.0,5.26,2.6,Glushon Sports Management
3,Nigel Hayes-Davis,"Jul 09, 2025",Free Agent,1.0,2.05,2.0,LIFT Sports Management
4,Bruce Brown Jr.,"Jul 09, 2025",Free Agent,1.0,3.08,3.1,CAA Sports
...,...,...,...,...,...,...,...
92,Shai Gilgeous-Alexander MGMT,NaN,Agency Total,4.0,272.43,68.1,Shai Gilgeous-Alexander MGMT
93,Shetellia Riley,NaN,Agency Total,3.0,118.47,39.5,Shetellia Riley
94,Verus Management,NaN,Agency Total,2.0,4.96,2.5,Verus Management
95,WME Sports,NaN,Agency Total,8.0,259.59,32.4,WME Sports


In [284]:
pd.set_option('display.max_rows', 100)
#Read in the URLs for agency logos
agency_urls = pd.read_csv('agency-urls.csv')
agency_urls.head(5)

,Agency,URL
0,CAA Sports,https://e7.pngegg.com/pngimages/963/437/png-cl...
1,Shai Gilgeous-Alexander MGMT,https://cdn.nba.com/headshots/nba/latest/1040x...
2,Excel Sports Management,https://media.licdn.com/dms/image/v2/C4E0BAQH8...
3,Priority Sports & Entertainment,https://media.licdn.com/dms/image/v2/D560BAQGI...
4,Klutch Sports Group,https://image4.owler.com/logo/klutch-sports-gr...


In [285]:
#Join the agency data with the URL data on the Agency name
new_agency_data = pd.merge(new_agency_data, agency_urls, how='left',left_on='Agency', right_on='Agency')

#Perform a self-join to have agency-level totals (for sorting purposes)
new_agency_data = pd.merge(new_agency_data,new_agency_data, how='left', left_on='Agency', right_on='Player')

#Drop extra columns and properly name joined total amount to 'Agency amount'
new_agency_data.drop(['Contract Type_y', 'Length (years)_y', 'Average Salary_y', 'Agency_y', 'Date_y', 'Player_y','URL_y'], axis=1, inplace=True)
new_agency_data.rename(columns={'Total Amount ($ million)_y':'Agency Amount ($ million)'}, inplace=True)

#Sort the values by Agency amount and player contract amount within agency
new_agency_data.sort_values(by=['Agency Amount ($ million)', 'Total Amount ($ million)_x'], ascending=[False, False], inplace=True)

#Remove 'Agency Amount' values when the row represents a player and not an agency
new_agency_data['Agency Level Amount ($ million)'] = np.where(new_agency_data['Agency Amount ($ million)'] == new_agency_data['Total Amount ($ million)_x'], new_agency_data['Agency Amount ($ million)'], np.NaN)

#Disginguish clearly for each row whether it represents a player or an agency with the Contract Type column
new_agency_data['Contract Type_x'] = np.where(new_agency_data['Contract Type_x'] == "Agency Total", new_agency_data['Contract Type_x'], "Player")

#Remove redundant rows for SGA and Kyrie Irving (self-represented)
new_agency_data = new_agency_data[(new_agency_data['Player_x'] != 'Shai Gilgeous-Alexander') & (new_agency_data['Player_x'] != 'Kyrie Irving')]

#Filter data for agencies to generate >=$25M dollars for their clients
new_agency_data = new_agency_data[new_agency_data['Agency Amount ($ million)'] >= 25]

new_agency_data.head(15)

,Player_x,Date_x,Contract Type_x,Length (years)_x,Total Amount ($ million)_x,Average Salary_x,Agency_x,URL_x,Agency Amount ($ million),Agency Level Amount ($ million)
77,CAA Sports,NaN,Agency Total,32.0,755.74,23.6,CAA Sports,https://e7.pngegg.com/pngimages/963/437/png-cl...,755.74,755.74
72,Jaren Jackson Jr.,"Jun 30, 2025",Player,5.0,240.00,48.0,CAA Sports,https://e7.pngegg.com/pngimages/963/437/png-cl...,755.74,NaN
66,Devin Booker,"Jul 09, 2025",Player,2.0,133.25,66.6,CAA Sports,https://e7.pngegg.com/pngimages/963/437/png-cl...,755.74,NaN
28,Myles Turner,"Jul 06, 2025",Player,4.0,108.86,27.2,CAA Sports,https://e7.pngegg.com/pngimages/963/437/png-cl...,755.74,NaN
64,Julius Randle,"Jun 29, 2025",Player,3.0,100.00,33.3,CAA Sports,https://e7.pngegg.com/pngimages/963/437/png-cl...,755.74,NaN
31,Nickeil Alexander-Walker,"Jul 06, 2025",Player,4.0,62.00,15.5,CAA Sports,https://e7.pngegg.com/pngimages/963/437/png-cl...,755.74,NaN
60,Santi Aldama,"Jun 30, 2025",Player,3.0,52.50,17.5,CAA Sports,https://e7.pngegg.com/pngimages/963/437/png-cl...,755.74,NaN
63,Ziaire Williams,"Jun 30, 2025",Player,2.0,12.00,6.0,CAA Sports,https://e7.pngegg.com/pngimages/963/437/png-cl...,755.74,NaN
35,D'Angelo Russell,"Jul 06, 2025",Player,2.0,11.65,5.8,CAA Sports,https://e7.pngegg.com/pngimages/963/437/png-cl...,755.74,NaN
29,Guerschon Yabusele,"Jul 06, 2025",Player,2.0,11.28,5.6,CAA Sports,https://e7.pngegg.com/pngimages/963/437/png-cl...,755.74,NaN


In [286]:
#Load advanced metrics data
adv_stats = pd.read_csv('nba-adv-stats.csv')
adv_stats

,nba_id,Team,Player,DPM_rank,EPM_rank,LEBRON_rank,consensus_rank,salary,cap_pct,Callouts,Unsigned
0,1628983,Oklahoma City Thunder,Shai Gilgeous-Alexander,3,1,1,1.666667,35859950,0.2551,Y,NaN
1,203999,Denver Nuggets,Nikola Jokic,2,2,2,2.000000,51415938,0.3657,NaN,NaN
2,203507,Milwaukee Bucks,Giannis Antetokounmpo,1,4,4,3.000000,48787676,0.3470,NaN,NaN
3,1628369,Boston Celtics,Jayson Tatum,5,8,3,5.333333,34848340,0.2479,NaN,NaN
4,1628378,Cleveland Cavaliers,Donovan Mitchell,7,7,8,7.333333,35410310,0.2519,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
434,1642275,Charlotte Hornets,Tidjane Salaun,495,497,555,515.666667,7488720,0.0533,NaN,NaN
435,203995,Phoenix Suns,Vasilije Micic,517,488,557,520.666667,7723000,0.0549,NaN,NaN
436,1642358,Washington Wizards,AJ Johnson,528,500,553,527.000000,2795294,0.0199,NaN,NaN
437,1641733,Charlotte Hornets,Nick Smith Jr.,533,513,565,537.000000,2587200,0.0184,NaN,NaN


In [287]:
#Left join free agent salaries to advanced stats
salary_stats_comp = pd.merge(adv_stats, free_agent_salaries,how='left', left_on='Player', right_index=True)

#Update the salary as a percentage of the cap to reflect new contract for free agents, and no change otherwise
salary_stats_comp['updated_cap_pct'] = np.where(pd.isna(salary_stats_comp['current_cap_pct']), salary_stats_comp['cap_pct'], salary_stats_comp['new_avg_cap_pct'])

#Distinguish between free agents and non-free agents (for data viz)
salary_stats_comp['Contract Type'] = np.where(pd.isna(salary_stats_comp['current_cap_pct']), 'other', 'free agent')

#Restrict analysis to the top 50 players by composite rank
salary_stats_comp.head(50)

,nba_id,Team,Player,DPM_rank,EPM_rank,LEBRON_rank,consensus_rank,salary,cap_pct,Callouts,...,Date,Contract Type,Length (years),Total Amount ($ million),Average Salary,current_salary ($ million),current_cap_pct,new_avg_cap_pct,difference,updated_cap_pct
0,1628983,Oklahoma City Thunder,Shai Gilgeous-Alexander,3,1,1,1.666667,35859950,0.2551,Y,...,"Jul 07, 2025",free agent,4.0,272.43,68.11,35.86,0.2551,0.3633,0.1082,0.3633
1,203999,Denver Nuggets,Nikola Jokic,2,2,2,2.000000,51415938,0.3657,NaN,...,NaN,other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.3657
2,203507,Milwaukee Bucks,Giannis Antetokounmpo,1,4,4,3.000000,48787676,0.3470,NaN,...,NaN,other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.3470
3,1628369,Boston Celtics,Jayson Tatum,5,8,3,5.333333,34848340,0.2479,NaN,...,NaN,other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2479
4,1628378,Cleveland Cavaliers,Donovan Mitchell,7,7,8,7.333333,35410310,0.2519,NaN,...,NaN,other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2519
5,1629029,Los Angeles Lakers,Luka Doncic,4,3,16,7.666667,43031940,0.3061,Y,...,NaN,other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.3061
6,1630169,Indiana Pacers,Tyrese Haliburton,12,9,6,9.000000,42176000,0.3000,NaN,...,NaN,other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.3000
7,201939,Golden State Warriors,Stephen Curry,10,6,15,10.333333,55761216,0.3966,NaN,...,NaN,other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.3966
8,1627826,Los Angeles Clippers,Ivica Zubac,16,10,9,11.666667,11743210,0.0835,NaN,...,NaN,other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0835
9,1630162,Minnesota Timberwolves,Anthony Edwards,11,22,7,13.333333,42176400,0.3000,NaN,...,NaN,other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.3000
